In [ ]:
import os
import glob
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm  # Dùng tqdm của notebook cho đẹp

# --- CẤU HÌNH HỆ THỐNG (HYPERPARAMETERS) ---
CONFIG = {
    "DATASET_DIR": "/kaggle/input/clean-noised-dataset/denoised_dataset_all",  # Thư mục chứa file .npy
    "OUTPUT_DIR": "/kaggle/working",            # Thư mục lưu model
    "BATCH_SIZE": 64,                       # Số lượng mẫu mỗi lần học
    "EPOCHS": 50,                           # Tổng số vòng học
    "LR": 1e-3,                             # Tốc độ học (Learning Rate)
    "TARGET_LEN": 1300,                     # Độ dài tín hiệu (10s * 130Hz)
    "SEED": 42                              # Random seed để tái lập kết quả
}

# Tạo thư mục lưu model nếu chưa có
os.makedirs(CONFIG["OUTPUT_DIR"], exist_ok=True)

# Chọn thiết bị (GPU ưu tiên)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"⚙️ Thiết bị đang sử dụng: {DEVICE}")

In [ ]:
class ECGDenoisingDataset(Dataset):
    def __init__(self, file_paths, augment=False):
        self.files = file_paths
        self.augment = augment

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        # Load file .npy (chứa dictionary {'input': ..., 'target': ...})
        try:
            data = np.load(self.files[idx], allow_pickle=True).item()
            x = data['input']   # Tín hiệu nhiễu
            y = data['target']  # Tín hiệu sạch (Ground Truth)
            
            # --- DATA AUGMENTATION (Tăng cường dữ liệu) ---
            # Giúp model học được nhiều tình huống lạ hơn
            if self.augment:
                # 1. Random Flip: Giả lập đeo ngược dây hoặc trục tim thay đổi
                if np.random.rand() > 0.5:
                    x = -x
                    y = -y
                
                # 2. Random Scale: Giả lập biên độ thay đổi do trở kháng da/mồ hôi
                # Tăng giảm biên độ từ 80% đến 120%
                scale = np.random.uniform(0.8, 1.2)
                x = x * scale
                y = y * scale

            # Chuyển sang Tensor PyTorch
            # Shape gốc: (1300,) -> Shape Model cần: (1, 1300) [Channel, Time]
            x_t = torch.from_numpy(x.copy()).float().unsqueeze(0)
            y_t = torch.from_numpy(y.copy()).float().unsqueeze(0)
            
            return x_t, y_t
            
        except Exception as e:
            print(f"Lỗi file {self.files[idx]}: {e}")
            # Trả về tensor rỗng để không crash (cần xử lý ở collate_fn nếu lỗi nhiều)
            return torch.zeros(1, CONFIG["TARGET_LEN"]), torch.zeros(1, CONFIG["TARGET_LEN"])

# --- Test thử Dataset ---
# Lấy danh sách file
all_files = sorted(glob.glob(os.path.join(CONFIG["DATASET_DIR"], "*.npy")))
print(f"📂 Tìm thấy tổng cộng {len(all_files)} file dữ liệu.")

if len(all_files) > 0:
    # Chia tập Train/Val
    train_files, val_files = train_test_split(all_files, test_size=0.1, random_state=CONFIG["SEED"])
    
    # Tạo Loader
    train_ds = ECGDenoisingDataset(train_files, augment=True)
    val_ds = ECGDenoisingDataset(val_files, augment=False)
    
    train_dl = DataLoader(train_ds, batch_size=CONFIG["BATCH_SIZE"], shuffle=True, num_workers=2)
    val_dl = DataLoader(val_ds, batch_size=CONFIG["BATCH_SIZE"], shuffle=False, num_workers=2)
    
    print(f"✅ Đã chia: Train ({len(train_files)}) | Val ({len(val_files)})")
else:
    print("❌ CẢNH BÁO: Không tìm thấy dữ liệu! Hãy kiểm tra lại đường dẫn.")

In [ ]:
# --- CÁC KHỐI CƠ BẢN (BUILDING BLOCKS) ---

class ConvBlock(nn.Module):
    """Khối tích chập kép: Conv -> BN -> ReLU -> Conv -> BN -> ReLU"""
    def __init__(self, in_c, out_c):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv1d(in_c, out_c, 3, padding=1, bias=False),
            nn.BatchNorm1d(out_c),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv1d(out_c, out_c, 3, padding=1, bias=False),
            nn.BatchNorm1d(out_c),
            nn.LeakyReLU(0.1, inplace=True)
        )
    def forward(self, x): return self.conv(x)

class AttentionGate(nn.Module):
    """
    Cổng chú ý (Attention Gate):
    Lọc bỏ nhiễu từ tín hiệu Skip Connection bằng cách so sánh với tín hiệu Gating từ layer sâu hơn.
    """
    def __init__(self, F_g, F_l, F_int):
        super().__init__()
        # W_g: Xử lý tín hiệu Gating (từ layer sâu hơn - thô hơn)
        self.W_g = nn.Sequential(
            nn.Conv1d(F_g, F_int, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm1d(F_int)
        )
        # W_x: Xử lý tín hiệu Skip (từ layer nông hơn - chi tiết hơn + nhiễu)
        self.W_x = nn.Sequential(
            nn.Conv1d(F_l, F_int, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm1d(F_int)
        )
        # Psi: Tính trọng số Attention (0 -> 1)
        self.psi = nn.Sequential(
            nn.Conv1d(F_int, 1, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm1d(1),
            nn.Sigmoid()
        )
        self.relu = nn.ReLU(inplace=True)

    def forward(self, g, x):
        g1 = self.W_g(g)
        x1 = self.W_x(x)
        psi = self.relu(g1 + x1)
        psi = self.psi(psi) # Map trọng số: 1 là giữ lại (tín hiệu tim), 0 là bỏ (nhiễu cơ)
        return x * psi 

# --- MODEL CHÍNH: ATTENTION U-NET ---
class AttentionUNet(nn.Module):
    def __init__(self):
        super().__init__()
        # --- Encoder (Downsampling) ---
        self.enc1 = ConvBlock(1, 32)
        self.pool1 = nn.MaxPool1d(2)
        self.enc2 = ConvBlock(32, 64)
        self.pool2 = nn.MaxPool1d(2)
        self.enc3 = ConvBlock(64, 128)
        self.pool3 = nn.MaxPool1d(2)
        self.enc4 = ConvBlock(128, 256)
        self.pool4 = nn.MaxPool1d(2)

        # --- Bottleneck (Nơi nén thông tin nhất) ---
        self.center = ConvBlock(256, 512)

        # --- Decoder (Upsampling) + Attention Gates ---
        # Level 4
        self.up4 = nn.ConvTranspose1d(512, 256, 2, stride=2)
        self.att4 = AttentionGate(F_g=256, F_l=256, F_int=128)
        self.dec4 = ConvBlock(512, 256) # 256(up) + 256(skip) = 512 input -> giảm về 256

        # Level 3
        self.up3 = nn.ConvTranspose1d(256, 128, 2, stride=2)
        self.att3 = AttentionGate(F_g=128, F_l=128, F_int=64)
        self.dec3 = ConvBlock(256, 128)

        # Level 2
        self.up2 = nn.ConvTranspose1d(128, 64, 2, stride=2)
        self.att2 = AttentionGate(F_g=64, F_l=64, F_int=32)
        self.dec2 = ConvBlock(128, 64)

        # Level 1
        self.up1 = nn.ConvTranspose1d(64, 32, 2, stride=2)
        self.att1 = AttentionGate(F_g=32, F_l=32, F_int=16)
        self.dec1 = ConvBlock(64, 32)

        # Output Layer
        self.final = nn.Conv1d(32, 1, 1) # Đưa về 1 kênh tín hiệu

    def forward(self, x):
        # Padding để chiều dài chia hết cho 16 (do 4 lần pooling: 2^4 = 16)
        # Nếu input 1300 -> pad thành 1312
        orig_len = x.shape[-1]
        pad_len = (16 - orig_len % 16) % 16
        if pad_len > 0: x = F.pad(x, (0, pad_len))
        
        # Encoder Path
        e1 = self.enc1(x)
        p1 = self.pool1(e1)
        e2 = self.enc2(p1)
        p2 = self.pool2(e2)
        e3 = self.enc3(p2)
        p3 = self.pool3(e3)
        e4 = self.enc4(p3)
        p4 = self.pool4(e4)

        # Bottleneck
        c = self.center(p4)

        # Decoder Path with Attention
        # D4
        d4 = self.up4(c)
        x4 = self.att4(g=d4, x=e4) # Lọc nhiễu
        d4 = torch.cat((x4, d4), dim=1)
        d4 = self.dec4(d4)

        # D3
        d3 = self.up3(d4)
        x3 = self.att3(g=d3, x=e3)
        d3 = torch.cat((x3, d3), dim=1)
        d3 = self.dec3(d3)

        # D2
        d2 = self.up2(d3)
        x2 = self.att2(g=d2, x=e2)
        d2 = torch.cat((x2, d2), dim=1)
        d2 = self.dec2(d2)

        # D1
        d1 = self.up1(d2)
        x1 = self.att1(g=d1, x=e1)
        d1 = torch.cat((x1, d1), dim=1)
        d1 = self.dec1(d1)

        out = self.final(d1)
        
        # Cắt bỏ phần padding thừa để trả về đúng kích thước gốc
        return out[..., :orig_len]

# Kiểm tra model
model_test = AttentionUNet()
dummy_input = torch.randn(2, 1, 1300)
output = model_test(dummy_input)
print(f"✅ Model Check: Input {dummy_input.shape} -> Output {output.shape}")

In [4]:
class HybridLoss(nn.Module):
    """
    Kết hợp giữa:
    1. Pixel-wise Loss (L1 + MSE): Để khớp hình dạng tổng thể.
    2. Gradient Loss: Để giữ độ sắc nét của các đỉnh sóng (tránh làm mờ đỉnh R).
    """
    def __init__(self):
        super().__init__()
        self.l1 = nn.L1Loss() 
        self.mse = nn.MSELoss()
        
    def gradient_loss(self, pred, target):
        # Tính đạo hàm bậc 1 (hiệu số giữa các điểm kề nhau)
        diff_pred = pred[..., 1:] - pred[..., :-1]
        diff_target = target[..., 1:] - target[..., :-1]
        return self.l1(diff_pred, diff_target)

    def forward(self, pred, target):
        # Loss hình dáng
        loss_pixel = 0.5 * self.l1(pred, target) + 0.5 * self.mse(pred, target)
        
        # Loss độ dốc (độ sắc nét)
        loss_grad = self.gradient_loss(pred, target)
        
        # Tổng hợp
        return loss_pixel + 0.5 * loss_grad

In [ ]:
# Khởi tạo
model = AttentionUNet().to(DEVICE)
criterion = HybridLoss().to(DEVICE)
optimizer = optim.AdamW(model.parameters(), lr=CONFIG["LR"], weight_decay=1e-4)

# Scheduler: Giảm learning rate nếu Loss không cải thiện sau 3 epoch
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=3, verbose=True
)

print("🚀 Bắt đầu quá trình huấn luyện...")
history = {'train_loss': [], 'val_loss': []}
best_val_loss = float('inf')

for epoch in range(CONFIG["EPOCHS"]):
    # --- TRAIN ---
    model.train()
    train_loss_accum = 0
    
    # Progress bar cho mỗi epoch
    loop = tqdm(train_dl, desc=f"Epoch {epoch+1}/{CONFIG['EPOCHS']}", leave=False)
    
    for x, y in loop:
        x, y = x.to(DEVICE), y.to(DEVICE)
        
        optimizer.zero_grad()
        pred = model(x) # Forward
        loss = criterion(pred, y) # Tính loss
        loss.backward() # Backprop
        
        # Gradient Clipping (Chống bùng nổ gradient)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        optimizer.step() # Update weights
        
        train_loss_accum += loss.item()
        loop.set_postfix(loss=loss.item()) # Hiển thị loss hiện tại trên thanh progress
    
    avg_train_loss = train_loss_accum / len(train_dl)
    
    # --- VALIDATION ---
    model.eval()
    val_loss_accum = 0
    with torch.no_grad():
        for x, y in val_dl:
            x, y = x.to(DEVICE), y.to(DEVICE)
            pred = model(x)
            val_loss = criterion(pred, y)
            val_loss_accum += val_loss.item()
            
    avg_val_loss = val_loss_accum / len(val_dl)
    
    # Lưu lịch sử
    history['train_loss'].append(avg_train_loss)
    history['val_loss'].append(avg_val_loss)
    
    # Điều chỉnh LR
    scheduler.step(avg_val_loss)
    
    # In kết quả
    print(f"Epoch {epoch+1}: Train Loss = {avg_train_loss:.5f} | Val Loss = {avg_val_loss:.5f}")
    
    # Lưu Model tốt nhất
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        save_path = os.path.join(CONFIG["OUTPUT_DIR"], "best_attention_unet.pth")
        torch.save(model.state_dict(), save_path)
        print(f"   💾 Đã lưu Best Model (Loss: {best_val_loss:.5f})")

print("\n🎉 HUẤN LUYỆN HOÀN TẤT!")

In [ ]:
# 1. Vẽ biểu đồ Loss
plt.figure(figsize=(10, 5))
plt.plot(history['train_loss'], label='Train Loss', color='blue')
plt.plot(history['val_loss'], label='Validation Loss', color='orange')
plt.title('Training & Validation Loss History')
plt.xlabel('Epochs')
plt.ylabel('Loss (Hybrid)')
plt.legend()
plt.grid(True)
plt.show()

# 2. Test thực tế trên 1 mẫu ngẫu nhiên từ tập Val
# Load lại model tốt nhất
best_model = AttentionUNet().to(DEVICE)
best_model.load_state_dict(torch.load(os.path.join(CONFIG["OUTPUT_DIR"], "best_attention_unet.pth")))
best_model.eval()

# Lấy 1 batch
x_val, y_val = next(iter(val_dl))
x_val = x_val.to(DEVICE)

with torch.no_grad():
    pred_val = best_model(x_val)

# Chuyển về CPU để vẽ
x_np = x_val.cpu().numpy()
y_np = y_val.cpu().numpy()
pred_np = pred_val.cpu().numpy()

# Vẽ 3 mẫu đầu tiên
num_samples = 3
plt.figure(figsize=(15, 10))
time_axis = np.arange(CONFIG["TARGET_LEN"]) / 130 # Trục thời gian (giây)

for i in range(num_samples):
    plt.subplot(num_samples, 1, i+1)
    
    # Vẽ Input (Nhiễu)
    plt.plot(time_axis, x_np[i, 0], color='gray', alpha=0.5, label='Input (Noisy)')
    
    # Vẽ Target (Sạch - Ground Truth)
    plt.plot(time_axis, y_np[i, 0], color='green', linestyle='--', linewidth=1.5, alpha=0.8, label='Target (Clean)')
    
    # Vẽ Output (AI Denoised)
    plt.plot(time_axis, pred_np[i, 0], color='red', linewidth=1.5, label='AI Output')
    
    plt.title(f'Sample {i+1}: Denoising Result')
    plt.ylabel('Amplitude')
    if i == 0: plt.legend(loc='upper right')
    plt.grid(True, alpha=0.3)

plt.xlabel('Time (s)')
plt.tight_layout()
plt.show()